In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

# 1. Data Loading and Preparation
def load_and_preprocess_data():
    df = pd.read_csv("merged_macro_commodity.csv", parse_dates=['date'])
    df.set_index('date', inplace=True)
    return df

def add_rolling_features(df, commodity_cols):
    """Add rolling features inspired by Prophet implementation"""
    for commodity in commodity_cols:
        df[f'{commodity}_3mo_ma'] = df[commodity].rolling(3, min_periods=1).mean()
        # df[f'{commodity}_12mo_ma'] = df[commodity].rolling(12, min_periods=6).mean()
    df.fillna(method='ffill')
    df.fillna(method='bfill')
    if df.isnull().values.any():
        raise ValueError("Data still contains NaN values after filling")
    return df

def add_lagged_features(df, feature_cols):
    """Add lagged macroeconomic features"""
    for feature in feature_cols:
        df[f'{feature}_lag3'] = df[feature].shift(3)
    df.fillna(method='ffill')
    df.fillna(method='bfill')
    if df.isnull().values.any():
        raise ValueError("Data still contains NaN values after filling")
    return df

# 2. Model Configuration
commodity_cols = ['Crude Oil Brent Price', 'Cocoa Price', 'Gold Price']
feature_cols = ['Revenue, excluding grants (% of GDP)', 
               'GDP (constant 2015 US$)', 
               'Employment to population ratio (15+, total %)']

# Commodity-specific parameters (extended from Prophet config)
commodity_params = {
    'Crude Oil Brent Price': {
        'lstm_units1': 256,
        'lstm_units2': 128,
        'dropout1': 0.3,
        'dropout2': 0.2,
        'learning_rate': 0.0005,
        'epochs': 150,
        'batch_size': 32,
        'floor': 20,
        'cap': 150
    },
    'Cocoa Price': {
        'lstm_units1': 128,
        'lstm_units2': 64,
        'dropout1': 0.2,
        'dropout2': 0.1,
        'learning_rate': 0.001,
        'epochs': 100,
        'batch_size': 16,
        'floor': 1,
        'cap': 10
    },
    'Gold Price': {
        'lstm_units1': 192,
        'lstm_units2': 96,
        'dropout1': 0.25,
        'dropout2': 0.15,
        'learning_rate': 0.00075,
        'epochs': 120,
        'batch_size': 24,
        'floor': 150,
        'cap': 2500
    }
}

def create_lstm_model(input_shape, commodity_name):
    """Create commodity-specific LSTM model"""
    params = commodity_params[commodity_name]
    
    model = Sequential([
        LSTM(params['lstm_units1'], 
             activation='tanh', 
             return_sequences=True, 
             input_shape=input_shape),
        Dropout(params['dropout1']),
        LSTM(params['lstm_units2'], activation='tanh'),
        Dropout(params['dropout2']),
        Dense(len(commodity_cols))  # Multi-output
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=params['learning_rate']),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# 3. Data Processing Pipeline
def prepare_data(df, n_steps=12):
    """Prepare data for LSTM with Prophet-inspired features"""
    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)
    
    # Create sequences
    X, y = [], []
    for i in range(len(scaled_data) - n_steps):
        X.append(scaled_data[i:i+n_steps, :])
        y.append(scaled_data[i+n_steps, :len(commodity_cols)])  # Predict commodities only
    
    return np.array(X), np.array(y), scaler

# 4. Training and Evaluation
def train_model(model, X_train, y_train, commodity_name):
    """Train with early stopping"""
    params = commodity_params[commodity_name]
    
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_split=0.2,
        verbose=1
    )
    
    return history

def evaluate_model(model, X_test, y_test, scaler):
    """Evaluate model performance"""
    # Predict and inverse transform
    y_pred = model.predict(X_test)
    dummy = np.zeros((len(y_pred), scaler.n_features_in_))
    dummy[:,:len(commodity_cols)] = y_pred
    y_pred = scaler.inverse_transform(dummy)[:,:len(commodity_cols)]
    
    # Inverse transform actuals
    dummy[:,:len(commodity_cols)] = y_test
    y_test_actual = scaler.inverse_transform(dummy)[:,:len(commodity_cols)]
    
    # Calculate metrics
    metrics = {}
    for i, col in enumerate(commodity_cols):
        # mae = mean_absolute_error(y_test_actual[:,i], y_pred[:,i])
        mape = np.mean(np.abs((y_test_actual[:,i] - y_pred[:,i]) / y_test_actual[:,i])) * 100
        rmse = np.sqrt(mean_squared_error(y_test_actual[:,i], y_pred[:,i]))
        metrics[col] = {'MAPE': mape, 'RMSE': rmse}
    
    return metrics, y_pred

# 5. Forecasting
def make_forecasts(model, X, n_steps, forecast_steps, scaler):
    """Generate recursive forecasts"""
    forecasts = []
    current_batch = X[-1:]  # Most recent sequence
    
    for _ in range(forecast_steps):
        # Predict next step
        current_pred = model.predict(current_batch, verbose=0)[0]
        forecasts.append(current_pred)
        
        # Update batch with prediction
        new_row = np.concatenate([
            current_pred,
            scaler.transform(df.iloc[-1:])[0, len(commodity_cols):]  # Latest features
        ]).reshape(1, 1, -1)
        
        current_batch = np.concatenate([
            current_batch[:, 1:, :],  # Remove oldest
            new_row  # Add new prediction
        ], axis=1)
    
    # Inverse transform
    dummy = np.zeros((len(forecasts), scaler.n_features_in_))
    dummy[:,:len(commodity_cols)] = forecasts
    return scaler.inverse_transform(dummy)[:,:len(commodity_cols)]

# 6. Visualization
def plot_results(commodity_idx, y_test, y_pred, forecasts, test_dates, forecast_dates):
    """Plot results in Prophet-style"""
    plt.figure(figsize=(14, 6))
    
    # Historical data
    plt.plot(df.index[:-len(y_test)], df.iloc[:-len(y_test), commodity_idx], 
             'b-', label='Historical')
    
    # Test actuals
    plt.plot(test_dates, y_test[:, commodity_idx], 
             'k-', label='Actual')
    
    # Test predictions
    plt.plot(test_dates, y_pred[:, commodity_idx], 
             'r-', label='Test Predictions', alpha=0.7)
    
    # Forecasts
    plt.plot(forecast_dates, forecasts[:, commodity_idx], 
             'go--', label='Forecast')
    
    plt.title(f'{commodity_cols[commodity_idx]} Forecast')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Main Execution
if __name__ == "__main__":
    # Load and prepare data
    df = load_and_preprocess_data()
    df = add_rolling_features(df, commodity_cols)
    df = add_lagged_features(df, feature_cols)
    
    # Prepare sequences
    n_steps = 12
    X, y, scaler = prepare_data(df, n_steps)
    
    # Train-test split (temporal)
    train_size = int(0.8 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Train models
    models = {}
    histories = {}
    
    for commodity in commodity_cols:
        print(f"\nTraining {commodity} model...")
        model = create_lstm_model((n_steps, X.shape[2]), commodity)
        history = train_model(model, X_train, y_train, commodity)
        models[commodity] = model
        histories[commodity] = history
        
        # Plot training history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.title(f'{commodity} Training History')
        plt.legend()
        plt.show()
    
    # Evaluate models
    test_dates = df.index[n_steps+train_size:-n_steps]
    all_metrics = {}
    
    for i, commodity in enumerate(commodity_cols):
        metrics, y_pred = evaluate_model(models[commodity], X_test, y_test, scaler)
        all_metrics[commodity] = metrics[commodity]  # Since we predict all commodities together
        
        print(f"\n{commodity} Test Metrics:")
        # print(f"MAE: {metrics[commodity]['MAE']:.2f}")
        print(f"MAPE: {metrics[commodity]['MAPE']:.2f}%")
        print(f"RMSE: {metrics[commodity]['RMSE']:.2f}")
    
    # Generate and plot forecasts
    forecast_steps = 12
    forecast_dates = pd.date_range(start=df.index[-1], periods=forecast_steps+1, freq='M')[1:]
    
    for i, commodity in enumerate(commodity_cols):
        forecasts = make_forecasts(models[commodity], X_test, n_steps, forecast_steps, scaler)
        plot_results(i, 
                   scaler.inverse_transform(y_test.reshape(-1, len(commodity_cols))), 
                   y_pred, 
                   forecasts, 
                   test_dates, 
                   forecast_dates)

/var/folders/rd/tqc_535n6bb2mkqrf4_7nf6w0000gn/T/ipykernel_12974/725386740.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill')
/var/folders/rd/tqc_535n6bb2mkqrf4_7nf6w0000gn/T/ipykernel_12974/725386740.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill')
/var/folders/rd/tqc_535n6bb2mkqrf4_7nf6w0000gn/T/ipykernel_12974/725386740.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill')
/var/folders/rd/tqc_535n6bb2mkqrf4_7nf6w0000gn/T/ipykernel_12974/725386740.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill')


ValueError: Data still contains NaN values after filling

In [5]:
df.head()

,"Revenue, excluding grants (% of GDP)",GDP (constant 2015 US$),"Employment to population ratio (15+, total %)",Crude Oil Brent Price,Cocoa Price,Gold Price,Crude Oil Brent Price_3mo_ma,Cocoa Price_3mo_ma,Gold Price_3mo_ma,"Revenue, excluding grants (% of GDP)_lag3",GDP (constant 2015 US$)_lag3,"Employment to population ratio (15+, total %)_lag3"
date,,,,,,,,,,,,
1991-01-01,14.597519,1.386282e+10,74.834000,23.65,1.2417,383.64,23.650000,1.241700,383.640000,NaN,NaN,NaN
1991-02-01,14.372773,1.391355e+10,74.690804,19.40,1.2147,363.83,21.525000,1.228200,373.735000,NaN,NaN,NaN
1991-03-01,14.148027,1.396124e+10,74.556498,19.45,1.2103,363.34,20.833333,1.222233,370.270000,NaN,NaN,NaN
1991-04-01,13.923281,1.400648e+10,74.430605,19.25,1.1458,358.38,19.366667,1.190267,361.850000,14.597519,1.386282e+10,74.834000
1991-05-01,13.698535,1.404985e+10,74.312644,19.30,1.0622,356.95,19.333333,1.139433,359.556667,14.372773,1.391355e+10,74.690804
